<a href="https://colab.research.google.com/github/antoineelmallah/olx_colab/blob/main/estudo_olx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Retrieve advertising data

In [ ]:
!pip install geobr

import geobr

import matplotlib.pyplot as plt
import pandas as pd
import io
import numpy as np
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
dados = pd.read_csv(io.BytesIO(uploaded['olx_advertising.csv']), sep=';')
dados.head()

In [ ]:
dados.dtypes

# 2. Pre-processing

In [ ]:
dados.info()

In [ ]:
dados.describe()

## 2.1. Split accessories into columns

In [ ]:
# Split accessory into columns and removing accessories column
accessories = [ acc.split(', ') for acc in dados['accessories'] if type(acc) == str ]
accessories = set([ acc for acc_list in accessories for acc in acc_list ])
for accessory in accessories:
  dados[accessory] = dados['accessories'].str.contains(f'\, { accessory }, ', regex=True)

dados.drop(['accessories'], axis='columns', inplace=True)

dados.head(5)

## 2.2. Fill empty values

In [ ]:
# Door field
# Extract door number from model field, if exists
dados['doors'].fillna(value=dados['model'].str.extract(r'^.*\s(\d)P\s.*$', expand=False).apply(lambda x : int(x) if not pd.isna(x) else None), inplace=True)
# Set mean for other empty values
door_mean = dados.describe()['doors']['mean']
dados['doors'].fillna(door_mean, inplace=True)

In [ ]:
# HP field
# Extract hp number from model field, if exists
dados['hp'].fillna(value=dados['model'].str.extract(r'^.*\s(\d\.\d)\s.*$', expand=False).apply(lambda x : float(x) if not pd.isna(x) else None), inplace=True)
# Set mean for other empty values
door_mean = dados.describe()['hp']['mean']
dados['hp'].fillna(door_mean, inplace=True)

In [ ]:
#def get_label_that_appear_in_text(text, labels, other):
#  for label in labels:
#    if label.lower() in text.lower():
#      return label
#  return other

In [ ]:
# Accessories
for accessory in accessories:
  dados[accessory].fillna(False, inplace=True)

In [ ]:
def fill_numeric_na_with_mean(dados: pd.DataFrame, field: str):
  dados[field].fillna(dados.describe()[field]['mean'], inplace=True)

In [ ]:
# Fill empty numeric values with mean
fill_numeric_na_with_mean(dados, 'latitude')
fill_numeric_na_with_mean(dados, 'longitude')
fill_numeric_na_with_mean(dados, 'hp')

## 2.3. Format columns

In [ ]:
# format date fields
dados['creation_date'] = pd.to_datetime(dados['creation_date'], format='%Y-%m-%d %H:%M:%S.%f').apply(lambda d : d.floor('D'))
dados['last_update_date'] = pd.to_datetime(dados['last_update_date'], format='%Y-%m-%d %H:%M:%S.%f').apply(lambda d : d.floor('D'))

In [ ]:
last_processing_date = dados['last_update_date'].max().floor('D')
last_processing_date

In [ ]:
# Create date related columns
dados['activated period'] = (dados['last_update_date'] - dados['creation_date']).dt.days
dados['closed'] = dados['last_update_date'] < last_processing_date

In [ ]:
# Formatting model column
dados['model'] = dados['model'].str.split(f'\s').apply(lambda x : ' '.join(x[:2]))

In [ ]:
def split_categoric_data_into_columns(dados: pd.DataFrame, field: str):
  columns = dados[dados[field].isna() == False][field].unique()
  result = pd.DataFrame(columns=columns)
  for column in columns:
    result[column] = dados[field] == column
  return result

In [ ]:
# split_categoric_data_into_columns(dados, 'category')
df = pd.concat([dados,
    split_categoric_data_into_columns(dados, 'model'),
    split_categoric_data_into_columns(dados, 'brand'),
    split_categoric_data_into_columns(dados, 'vehicle_type'),
    split_categoric_data_into_columns(dados, 'fuel'),
    split_categoric_data_into_columns(dados, 'gear'),
    split_categoric_data_into_columns(dados, 'color'),
    split_categoric_data_into_columns(dados, 'steering')],
    axis='columns')
df.drop(columns=['model', 'brand', 'vehicle_type', 'fuel', 'gear', 'color', 'steering'], inplace=True)

In [ ]:
# calculating price variation
df['price variation'] = df['last_price'] - df['first_price']
df.drop(columns=['first_price', 'last_price'], inplace=True)

## 2.4. Removing not closed advertising

In [ ]:
threshold = 5
df = df[(df['closed'] == True) | (df['activated period'] > threshold)]
df.reset_index(drop=True, inplace=True)
df.head(5)

In [ ]:
df.shape

## 2.5. Removing unused columns

## 3. Random forest

In [ ]:
# Verification after pre-processing
df.describe()

In [ ]:
y = df['activated period'] <= threshold

In [ ]:
columns_to_keep = [ column for column in df.columns if column not in {
    'id',
    'code',
    'category',
    'creation_date',
    'last_update_date',
    'city',
    'activated period',
    'closed'} ]
X = df[columns_to_keep]

In [ ]:
# normalization
from sklearn import preprocessing
import numpy as np

cols = X.columns
scaler = preprocessing.MinMaxScaler(feature_range=(0, 2))
X = pd.DataFrame(scaler.fit_transform(X), columns=cols)
X

In [ ]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

skb = SelectKBest(chi2, k=10)
skb.fit(X, y)
cols_idxs = skb.get_support(indices=True)
X = X.iloc[:, cols_idxs]

In [ ]:
# correlation analysis
correlation = X.corr()
plot = sns.heatmap(correlation, annot=True, fmt='.1f', linewidths=.6)
plot

In [ ]:
# Split train and test data
from sklearn.model_selection import train_test_split

SEED = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, # numero de árvores de decisão (default=100)
                             max_depth=15, # número de níveis máximo que cada árvore de decisão deve ter
                             random_state=SEED)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
clf.feature_importances_

In [ ]:
importances = pd.Series(data=clf.feature_importances_, index=X.columns)
sns.barplot(x=importances, y=importances.index, orient='h').set_title('Importância de cada feature')


In [ ]:
print('\n** Analise do modelo')
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d').set_title('Short time advertising - confusion matrix (0, 1)')
print(classification_report(y_test,y_pred))


In [ ]:
# Carros mais anunciados
dados[['model', 'year', 'id']]\
  .groupby(by=['model', 'year'])\
  .count()\
  .sort_values(by='id', ascending=False)\
  .head(10)

In [ ]:
# Carros mais vendidos
dados[(dados['closed']) | (dados['last_update_date'] < last_processing_date)][['model', 'year', 'id']]\
  .groupby(by=['model', 'year'])\
  .count()\
  .sort_values(by='id', ascending=False)\
  .head(10)

In [ ]:
# Carros com maior variação de preço no anuncio
dados['price_variation'] = dados['last_price'] - dados['first_price']
dados[['model', 'year', 'first_price', 'price_variation']]\
  .groupby(by=['model', 'year'])\
  .mean()\
  .sort_values(by='price_variation', ascending=False)\
  .head(10)

In [ ]:
all_muni = geobr.read_municipality(code_muni="RJ", year=2022)

dados_per_city = dados[['city', 'price_variation']].groupby(by='city').mean().reset_index()

all_muni['name_muni'] = all_muni['name_muni'].str.lower()
dados_per_city['city'] = dados_per_city['city'].str.lower()

all_muni = all_muni.merge(dados_per_city, how='left', left_on='name_muni', right_on='city')

In [ ]:
#all_muni = geobr.read_municipality(code_muni="RJ", year=2010)

#all_muni['name_muni'] = all_muni['name_muni'].str.lower()
#dados['city'] = dados['city'].str.lower()

#all_muni = all_muni.merge(dados, how='left', left_on='name_muni', right_on='city')

fig, ax = plt.subplots(figsize=(15, 15), dpi=300)

all_muni.plot(
    column='price_variation',
    #facecolor="#2D3E50",
    edgecolor="#000000",
    cmap="viridis",
    legend=True,
    legend_kwds={
        "label": "Price variation",
        "orientation": "horizontal",
        "shrink": 0.6,
    },
    ax=ax)

ax.set_title("Municipalities of Rio de Janeiro, 2022", fontsize=20)
ax.axis("off")